Experimentation on Streaming LLM with attention sinks.
Code taken from https://github.com/mit-han-lab/streaming-llm.

In [ ]:
!git clone https://github.com/mit-han-lab/streaming-llm.git

Cloning into 'streaming-llm'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 104 (delta 27), reused 26 (delta 21), pack-reused 69
Receiving objects: 100% (104/104), 39.20 MiB | 60.00 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
pip install transformers==4.33.0 accelerate datasets evaluate wandb scikit-learn scipy sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%cd /content/streaming-llm/
!python setup.py develop

/content/streaming-llm
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other s

In [ ]:
!pip install flash-attn --no-build-isolation
!pip install git+https://github.com/HazyResearch/flash-attention.git#subdirectory=csrc/rotary

In [ ]:
!ldconfig /usr/lib64-nvidia

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link



In [ ]:
!export PYTHONPATH=/content/streaming-llm:$PYTHONPATH

In [ ]:
class Namespace(object):

    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

In [ ]:
import torch
from tqdm import tqdm
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from torch.nn import CrossEntropyLoss
from streaming_llm.kv_cache import StartRecentKVCache
from streaming_llm.utils import parse_args, load

device = "cuda"

args = Namespace(
    model_name_or_path='NousResearch/Llama-2-7b-hf',  #NousResearch/Yarn-Llama-2-13b-64k
    revision='main',
    tokenizer_name_or_path=None,
    dataset_name='pg19',
    task=None,
    split='test',
    num_samples=1,
    output_dir='outputs/debug',
    enable_start_recent_kv_cache=True,
    start_size=1,
    recent_size=255,
    enable_pos_shift=True,
    num_eval_tokens=1024 + 32,
    repeat_tokens=True,
    repeat_step=1,
)

# data = load_dataset(
#     args.dataset_name, args.task, split=args.split, streaming=True
# ).take(args.num_samples)

# model, tokenizer = load(args.model_name_or_path)

In [ ]:
nlls = []
full_attn_nlls = []
sliding_nlls = []
offsets = []
ppls = []
full_attn_ppls = []
sliding_ppls = []

loss_fn = CrossEntropyLoss(reduction="none")
past_key_values = None

if args.enable_start_recent_kv_cache:
    if "llama" in model.config.model_type:
        k_seq_dim = v_seq_dim = 2
    else:
        raise ValueError(f"got {model.config.model_type}")

if args.enable_pos_shift:
    if "llama" in model.config.model_type:
        from streaming_llm.pos_shift.modify_llama import enable_llama_pos_shift_attention

        enable_llama_pos_shift_attention(model)
    else:
        raise ValueError(f"got {model.config.model_type}")


os.makedirs(args.output_dir, exist_ok=True)
f = open(f"{args.output_dir}/log.txt", "w")

for text in map(lambda x: x['text'], data):
    num_eval_tokens = 0
    encodings = tokenizer(text, return_tensors="pt")
    # tok_idx_offset = 1000
    # encodings.input_ids = torch.arange(tok_idx_offset, args.num_eval_tokens + tok_idx_offset, dtype=encodings.input_ids.dtype)[None]

    seq_len = encodings.input_ids.size(1)
    repeat_toks = args.start_size + args.recent_size
    encodings.input_ids = encodings.input_ids[:,:args.num_eval_tokens]
    assert repeat_toks < args.num_eval_tokens

    for j in range(0, args.num_eval_tokens - 2 * repeat_toks, args.repeat_step):
        curr_input_ids = encodings.input_ids[:,:2*repeat_toks + j].clone()
        eval_tok_shift = repeat_toks + j
        print(f"step={j//args.repeat_step+1}/{(args.num_eval_tokens - 2 * repeat_toks) // args.repeat_step}")

        # repeat tokens - offset repetition by j
        if args.repeat_tokens:
            curr_input_ids[:,repeat_toks + j:] = curr_input_ids[:,:repeat_toks]

        # sliding-window attn (with recomputation)
        pbar = tqdm(range(eval_tok_shift, 2*repeat_toks + j - 1))

        #print(f"evaluating windowed attn on tokens {eval_tok_shift - repeat_toks + 1}-{2*repeat_toks + j}")

        for idx in pbar:
            input_ids = curr_input_ids[:, max(0,idx-repeat_toks):idx].to(device)
            with torch.no_grad():
                #print(f"windowed: input_ids={input_ids}")
                outputs = model(
                    input_ids
                )
                logits = outputs.logits[:,-1:].view(-1, model.config.vocab_size)
                past_key_values = outputs.past_key_values
                label = curr_input_ids[:, idx: idx + 1].to(logits.device).view(-1)
                #print(f"windowed: labels={label}")
                neg_log_likelihood = loss_fn(logits, label)
                sliding_nlls.append(neg_log_likelihood)

        pbar.close()

        # full-attn LLM
        input_ids = curr_input_ids[:, :2*repeat_toks + j].to(device)
        #print(f"evaluating full attn on tokens {input_ids.shape[1]-repeat_toks+1}-{2*repeat_toks + j}")
        with torch.no_grad():
            outputs = model(
                input_ids,
            )
            #print(f"full-attn: input_ids={input_ids}")
            logits = outputs.logits[:,-repeat_toks-1:-1].view(-1, model.config.vocab_size)
            label = input_ids[:,-repeat_toks:].to(logits.device).view(-1)
            #print(f"full-attn: label={label}")
            neg_log_likelihood = loss_fn(logits, label)
            full_attn_nlls.append(neg_log_likelihood)

        # streaming LLM - process initial cache warmup in parallel
        input_ids = curr_input_ids[:, :repeat_toks].to(device)
        #print(f"evaluating streaming on tokens {max(repeat_toks, eval_tok_shift)+1}-{2*repeat_toks + j}")
        kv_cache = StartRecentKVCache(
            start_size=args.start_size,
            recent_size=args.recent_size,
            k_seq_dim=k_seq_dim,
            v_seq_dim=v_seq_dim,
        )
        #print(f"stream init: input_ids={input_ids}")
        with torch.no_grad():
            outputs = model(
                input_ids,
                past_key_values=past_key_values,
                use_cache=True,
            )
            past_key_values = outputs.past_key_values
            if kv_cache is not None:
                past_key_values = kv_cache(past_key_values)
        if repeat_toks >= eval_tok_shift:
            logits = outputs.logits[:,-1:].view(-1, model.config.vocab_size)
            label = curr_input_ids[:, repeat_toks : repeat_toks + 1].to(logits.device).view(-1)
            #print(f"stream init: label={label}")
            neg_log_likelihood = loss_fn(logits, label)

        # begin streaming
        pbar = tqdm(range(repeat_toks, 2*repeat_toks + j - 1))

        for idx in pbar:
            input_ids = curr_input_ids[:, idx : idx + 1].to(device)
            with torch.no_grad():
                outputs = model(
                    input_ids,
                    past_key_values=past_key_values,
                    use_cache=True,
                )
                #print(f"streaming: input_ids={input_ids}")
                logits = outputs.logits.view(-1, model.config.vocab_size)
                past_key_values = outputs.past_key_values
                label = curr_input_ids[:, idx + 1 : idx + 2].to(logits.device).view(-1)
                neg_log_likelihood = loss_fn(logits, label)
                if kv_cache is not None:
                    past_key_values = kv_cache(past_key_values)
            if idx >= eval_tok_shift:
                nlls.append(neg_log_likelihood)
                #print(f"streaming: label={label}")
            pbar.set_description(
                f"nll: {neg_log_likelihood.item():.2f}, ppl: {torch.exp(neg_log_likelihood).item():.2f}"
            )
            #print(neg_log_likelihood.item(), file=f, flush=True)

        pbar.close()

        offsets.append(j)
        ppls.append(torch.exp(torch.cat(nlls).mean()).item())
        full_attn_ppls.append(torch.exp(torch.cat(full_attn_nlls).mean()).item())
        sliding_ppls.append(torch.exp(torch.stack(sliding_nlls).mean()).item())


f.close()

step=1/544


100%|██████████| 255/255 [00:11<00:00, 23.13it/s]


StartRecentKVCache: 1, 255


nll: 0.17, ppl: 1.18: 100%|██████████| 255/255 [00:11<00:00, 22.06it/s]


step=2/544


...

nll: 0.29, ppl: 1.34: 100%|██████████| 280/280 [00:12<00:00, 22.09it/s]


step=27/544


100%|██████████| 255/255 [00:11<00:00, 23.18it/s]


StartRecentKVCache: 1, 255


nll: 1.06, ppl: 2.89:  80%|████████  | 225/281 [00:10<00:02, 22.44it/s]

In [ ]:
print(full_attn_ppls)
print(ppls)
print(sliding_ppls)

[1.402909755706787, 1.4052437543869019, 1.404779076576233, 1.4090758562088013, 1.4134364128112793, 1.4172700643539429, 1.4204535484313965, 1.4195637702941895, 1.4187251329421997, 1.4263849258422852, 1.4335627555847168]
[2.718900203704834, 4.016863822937012, 4.5563812255859375, 4.914188385009766, 5.1250128746032715, 5.25825309753418, 5.360547065734863, 5.44130802154541, 5.530566692352295, 5.591329097747803, 5.649539947509766]
[5.766645908355713, 6.2545881271362305, 6.386405944824219, 6.479013442993164, 6.538914680480957, 6.567478179931641, 6.552601337432861, 6.5352325439453125, 6.539529800415039, 6.537642002105713, 6.588767051696777]


In [ ]:
logits.shape, label.shape

(torch.Size([8, 32000]), torch.Size([1]))

In [ ]:
curr_input_ids[:, max(0,idx-repeat_toks):idx+1]

tensor([[1006, 1007, 1000, 1001, 1002, 1003, 1004, 1005, 1006]])

In [ ]:
curr_input_ids

tensor([[1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1000, 1001, 1002, 1003,
         1004, 1005, 1006, 1007]])

In [ ]:
list(range(0, args.num_eval_tokens - 2 * repeat_toks, args.repeat_step))

[0, 128, 256, 384, 512, 640, 768, 896]

In [ ]:
print(ppls)
print(full_attn_ppls)
print(sliding_ppls)

[1.509871244430542, 84.67305755615234, 396.76214599609375, 1057.8154296875, 1654.1778564453125, 2369.919677734375, 3034.802001953125, 3964.217529296875, 4848.52197265625, 5820.080078125, 6702.83349609375, 7562.923828125, 8321.2314453125, 9160.236328125, 9860.6015625, 10629.921875]
[6.709757328033447, 6.241941928863525, 6.940530300140381, 6.778741359710693, 6.974102020263672, 7.429309368133545, 6.702706336975098, 6.80616569519043, 6.346508502960205, 6.005157470703125, 5.8393731117248535, 5.620218753814697, 5.47456693649292, 5.294277667999268, 5.3683013916015625, 5.2660627365112305]
[6717.744140625, 12659.8623046875, 18004.517578125, 18762.900390625, 21132.140625, 21226.466796875, 20000.583984375, 22740.46875, 23553.732421875, 25710.1015625, 26982.345703125, 27379.380859375, 28355.13671875, 29006.0859375, 30106.89453125, 30694.435546875]


In [ ]:
import torch
from tqdm import tqdm
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from torch.nn import CrossEntropyLoss
from streaming_llm.kv_cache import StartRecentKVCache
from streaming_llm.utils import parse_args, load

device = "cuda"

args = Namespace(
    model_name_or_path='NousResearch/Llama-2-7b-hf',  #NousResearch/Yarn-Llama-2-13b-64k
    revision='main',
    tokenizer_name_or_path=None,
    dataset_name='pg19',
    task=None,
    split='test',
    num_samples=1,
    output_dir='outputs/debug',
    enable_start_recent_kv_cache=True,
    start_size=1,
    recent_size=7,
    enable_pos_shift=True,
    num_eval_tokens=16,
    repeat_tokens=False,
)

# data = load_dataset(
#     args.dataset_name, args.task, split=args.split, streaming=True
# ).take(args.num_samples)

#model, tokenizer = load(args.model_name_or_path)

nlls = []
full_attn_nlls = []
sliding_nlls = []

loss_fn = CrossEntropyLoss(reduction="none")
past_key_values = None

if args.enable_start_recent_kv_cache:
    if "llama" in model.config.model_type:
        k_seq_dim = v_seq_dim = 2
    else:
        raise ValueError(f"got {model.config.model_type}")
    kv_cache = StartRecentKVCache(
        start_size=args.start_size,
        recent_size=args.recent_size,
        k_seq_dim=k_seq_dim,
        v_seq_dim=v_seq_dim,
    )
else:
    kv_cache = None

if args.enable_pos_shift:
    if "llama" in model.config.model_type:
        from streaming_llm.pos_shift.modify_llama import enable_llama_pos_shift_attention

        enable_llama_pos_shift_attention(model)
    else:
        raise ValueError(f"got {model.config.model_type}")


os.makedirs(args.output_dir, exist_ok=True)
f = open(f"{args.output_dir}/log.txt", "w")

num_eval_tokens = 0
for text in map(lambda x: x['text'], data):
    encodings = tokenizer(text, return_tensors="pt")
    encodings.input_ids = torch.arange(0, 8, dtype=encodings.input_ids.dtype).repeat(2)[None]

    #print(encodings.input_ids[:, :10])

    seq_len = encodings.input_ids.size(1)
    eval_tok_shift = args.start_size + args.recent_size

    # repeat tokens
    if args.repeat_tokens:
        encodings.input_ids[:,eval_tok_shift:2*eval_tok_shift] = encodings.input_ids[:,:eval_tok_shift]

    #print(f"seq_len: {seq_len}")

    # sliding-window attn (with recomputation)
    pbar = tqdm(range(eval_tok_shift, args.num_eval_tokens - 1))

    for idx in pbar:
        input_ids = encodings.input_ids[:, max(0,idx-eval_tok_shift):idx+1].to(device)
        with torch.no_grad():
            outputs = model(
                input_ids
            )
            logits = outputs.logits[:,-1:].view(-1, model.config.vocab_size)
            past_key_values = outputs.past_key_values
            label = encodings.input_ids[:, idx + 1 : idx + 2].to(logits.device).view(-1)
            print(f"windowed: input_ids={input_ids}")
            print(f"windowed: label={label}")
            neg_log_likelihood = loss_fn(logits, label)
            sliding_nlls.append(neg_log_likelihood)

    pbar.close()

    # full-attn LLM
    input_ids = encodings.input_ids[:, :args.num_eval_tokens].to(device)
    with torch.no_grad():
        outputs = model(
            input_ids,
        )
        logits = outputs.logits[:,eval_tok_shift:-1].view(-1, model.config.vocab_size)
        label = input_ids[:,eval_tok_shift+1:].to(logits.device).view(-1)
        print(f"full: input_ids={input_ids}")
        print(f"full: label={label}")
        neg_log_likelihood = loss_fn(logits, label)
        full_attn_nlls.append(neg_log_likelihood)

    # streaming LLM - process initial cache warmup in parallel
    input_ids = encodings.input_ids[:, :eval_tok_shift].to(device)
    with torch.no_grad():
        outputs = model(
            input_ids,
            past_key_values=past_key_values,
            use_cache=True,
        )
        print(f"stream init: input_ids={input_ids}")
        past_key_values = outputs.past_key_values
        if kv_cache is not None:
            past_key_values = kv_cache(past_key_values)

    # begin streaming
    pbar = tqdm(range(eval_tok_shift, args.num_eval_tokens - 1))

    for idx in pbar:
        input_ids = encodings.input_ids[:, idx : idx + 1].to(device)
        with torch.no_grad():
            outputs = model(
                input_ids,
                past_key_values=past_key_values,
                use_cache=True,
            )
            logits = outputs.logits.view(-1, model.config.vocab_size)
            past_key_values = outputs.past_key_values
            label = encodings.input_ids[:, idx + 1 : idx + 2].to(logits.device).view(-1)
            print(f"streaming: input_ids={input_ids}")
            neg_log_likelihood = loss_fn(logits, label)
            if kv_cache is not None:
                past_key_values = kv_cache(past_key_values)
        if idx >= eval_tok_shift:
            nlls.append(neg_log_likelihood)
            print(f"streaming: label={label}")
        pbar.set_description(
            f"nll: {neg_log_likelihood.item():.2f}, ppl: {torch.exp(neg_log_likelihood).item():.2f}"
        )
        print(neg_log_likelihood.item(), file=f, flush=True)
        num_eval_tokens += 1
        if args.num_eval_tokens is not None and num_eval_tokens >= args.num_eval_tokens:
            break

    if args.num_eval_tokens is not None and num_eval_tokens >= args.num_eval_tokens:
        break

f.close()

ppl = torch.exp(torch.stack(nlls).mean()).item()
full_attn_ppl = torch.exp(torch.cat(full_attn_nlls).mean()).item()
sliding_ppl = torch.exp(torch.stack(sliding_nlls).mean()).item()

StartRecentKVCache: 1, 7


 43%|████▎     | 3/7 [00:00<00:00, 22.45it/s]

windowed: input_ids=tensor([[0, 1, 2, 3, 4, 5, 6, 7, 0]], device='cuda:0')
windowed: label=tensor([1], device='cuda:0')
windowed: input_ids=tensor([[1, 2, 3, 4, 5, 6, 7, 0, 1]], device='cuda:0')
windowed: label=tensor([2], device='cuda:0')
windowed: input_ids=tensor([[2, 3, 4, 5, 6, 7, 0, 1, 2]], device='cuda:0')
windowed: label=tensor([3], device='cuda:0')
windowed: input_ids=tensor([[3, 4, 5, 6, 7, 0, 1, 2, 3]], device='cuda:0')
windowed: label=tensor([4], device='cuda:0')
windowed: input_ids=tensor([[4, 5, 6, 7, 0, 1, 2, 3, 4]], device='cuda:0')
windowed: label=tensor([5], device='cuda:0')


100%|██████████| 7/7 [00:00<00:00, 22.52it/s]


windowed: input_ids=tensor([[5, 6, 7, 0, 1, 2, 3, 4, 5]], device='cuda:0')
windowed: label=tensor([6], device='cuda:0')
windowed: input_ids=tensor([[6, 7, 0, 1, 2, 3, 4, 5, 6]], device='cuda:0')
windowed: label=tensor([7], device='cuda:0')
full: input_ids=tensor([[0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7]], device='cuda:0')
full: label=tensor([1, 2, 3, 4, 5, 6, 7], device='cuda:0')
stream init: input_ids=tensor([[0, 1, 2, 3, 4, 5, 6, 7]], device='cuda:0')


...

streaming: input_ids=tensor([[1]], device='cuda:0')
streaming: label=tensor([2], device='cuda:0')
streaming: input_ids=tensor([[2]], device='cuda:0')
streaming: label=tensor([3], device='cuda:0')
streaming: input_ids=tensor([[3]], device='cuda:0')
streaming: label=tensor([4], device='cuda:0')
streaming: input_ids=tensor([[4]], device='cuda:0')
streaming: label=tensor([5], device='cuda:0')


nll: 0.60, ppl: 1.81:  86%|████████▌ | 6/7 [00:00<00:00, 21.60it/s]

streaming: input_ids=tensor([[5]], device='cuda:0')
streaming: label=tensor([6], device='cuda:0')


nll: 1.24, ppl: 3.46: 100%|██████████| 7/7 [00:00<00:00, 21.52it/s]

streaming: input_ids=tensor([[6]], device='cuda:0')
streaming: label=tensor([7], device='cuda:0')


In [ ]:
# partial forward propagation of information from remaining keys in rolling cache
sliding_ppl, ppl, full_attn_ppl

(4.56148624420166, 2.8147730827331543, 1.184415340423584)

In [ ]:
# without repeat
ppl, full_attn_ppl

(5.599760055541992, 5.257948398590088)

In [ ]:
# with repeat
ppl, full_attn_ppl

(2.5983119010925293, 1.184415340423584)